# Probability

It is hard to do data science without some sort of understanding of _probability_ and its mathematics. As with our treatment of statistics in Chapter 5, we’ll wave our hands a lot and elide many of the technicalities.

For our purposes you should think of probability as a way of quantifying the uncertainty associated with _events_ chosen from a some _universe_ of events. Rather than getting technical about what these terms mean, think of rolling a die. The universe consists of all possible outcomes. And any subset of these outcomes is an event; for example, “the die rolls a one” or “the die rolls an even number.”

Notationally, we write $P(E)$ to mean “the probability of the event $E$.”

We’ll use probability theory to build models. We’ll use probability theory to evaluate models. We’ll use probability theory all over the place.

One could, were one so inclined, get really deep into the philosophy of what probability theory _means_. (This is best done over beers.) We won’t be doing that.

# Dependence and Independence

Roughly speaking, we say that two events $E$ and $F$ are _dependent_ if knowing something about whether $E$ happens gives us information about whether $F$ happens (and vice versa). Otherwise they are _independent_.

For instance, if we flip a fair coin twice, knowing whether the first flip is Heads gives us no information about whether the second flip is Heads. These events are independent. On the other hand, knowing whether the first flip is Heads certainly gives us information about whether both flips are Tails. (If the first flip is Heads, then definitely it’s not the case that both flips are Tails.) These two events are dependent.

Mathematically, we say that two events $E$ and $F$ are independent if the probability that they both happen is the product of the probabilities that each one happens:

$$ P(E,F) = P(E)P(F) $$

In the example above, the probability of “first flip Heads” is 1/2, and the probability of “both flips Tails” is 1/4, but the probability of “first flip Heads _and_ both flips Tails” is 0.

# Conditional Probability

When two events $E$ and $F$ are independent, then by definition we have:

$$ P(E,F) = P(E)P(F) $$

If they are not necessarily independent (and if the probability of $F$ is not zero), then we define the probability of $E$ “conditional on $F$” as:

$$ P(E|F) = \frac{P(E,F)}{P(F)} $$

You should think of this as the probability that $E$ happens, given that we know that $F$ happens.

We often rewrite this as:

$$ P(E,F) = P(E|F)P(F) $$

When $E$ and $F$ are independent, you can check that this gives:

$$ P(E|F) = P(E) $$

which is the mathematical way of expressing that knowing $F$ occurred gives us no additional information about whether $E$ occurred.

One common tricky example involves a family with two (unknown) children.

If we assume that:

1. Each child is equally likely to be a boy or a girl
2. The gender of the second child is independent of the gender of the first child then the event “no girls” has probability 1/4, the event “one girl, one boy” has probability 1/2, and the event “two girls” has probability 1/4.

Now we can ask what is the probability of the event “both children are girls” ($B$) conditional on the event “the older child is a girl” ($G$)? Using the definition of conditional probability:

$$ P(B|G) = \frac{P(B,G)}{P(G)} = \frac{P(B)}{P(G)} = \frac{1}{2}$$

since the event $B$ and $G$ (“both children are girls _and_ the older child is a girl”) is just the event $B$. (Once you know that both children are girls, it’s necessarily true that the older child is a girl.)

Most likely this result accords with your intuition.

We could also ask about the probability of the event “both children are girls” conditional on the event “at least one of the children is a girl” ($L$). Surprisingly, the answer is different from before!

As before, the event $B$ and $L$ (“both children are girls _and_ at least one of the children is a girl”) is just the event $B$. This means we have:

$$ P(B|L) = \frac{P(B,L)}{P(L)} = \frac{P(B)}{P(L)} = \frac{1}{3} $$

How can this be the case? Well, if all you know is that at least one of the children is a girl, then it is twice as likely that the family has one boy and one girl than that it has both girls.

We can check this by “generating” a lot of families:

In [ ]:
def random_kid():
    return random.choice(["boy", "girl"])
both_girls = 0
older_girl = 0
either_girl = 0
random.seed(0)
for _ in range(10000):
    younger = random_kid()
    older = random_kid()
    if older == "girl":
        older_girl += 1
    if older == "girl" and younger == "girl":
        both_girls += 1
    if older == "girl" or younger == "girl":
        either_girl += 1
print "P(both | older):", both_girls / older_girl         # 0.514 ~ 1/2
print "P(both | either): ", both_girls / either_girl      # 0.342 ~ 1/3

# Bayes’s Theorem

One of the data scientist’s best friends is Bayes’s Theorem, which is a way of “reversing” conditional probabilities. Let’s say we need to know the probability of some event $E$ conditional on some other event $F$ occurring. But we only have information about the probability of $F$ conditional on $E$ occurring. Using the definition of conditional probability twice tells us that:

$$ P(E|F) = \frac{P(E,F)}{P(F)} = \frac{P(F|E)P(E)}{P(F)} $$

The event $F$ can be split into the two mutually exclusive events “$F$ and $E$” and “$F$ and not $E$.” If we write $\neg E$ for "not $E$ " (i.e., " $E$ doesn’t happen"), then:

$$ P(F) = P(F,E) = P(F,\neg E) $$

so that:

$$ P(E|F) = \frac{P(F|E)P(E)}{[P(F|E)P(E) + P(F|\neg E)P(\neg E)]} $$

which is how Bayes’s Theorem is often stated.

This theorem often gets used to demonstrate why data scientists are smarter than doctors. Imagine a certain disease that affects 1 in every 10,000 people. And imagine that there is a test for this disease that gives the correct result (“diseased” if you have the disease, “nondiseased” if you don’t) 99% of the time.

What does a positive test mean? Let’s use $T$ for the event “your test is positive” and $D$ for the event “you have the disease.” Then Bayes’s Theorem says that the probability that you have the disease, conditional on testing positive, is:

$$ P(D|T) = \frac{P(T|D)P(D)}{[P(T|D)P(D) + P(T|\neg D)P(\neg D)]} $$

Here we know that $P(T|D)$, the probability that someone with the disease tests positive, is 0.99. $P(D)$ , the probability that any given person has the disease, is 1/10,000 = 0.0001. $P(T|\neg D)$ , the probability that someone without the disease tests positive, is 0.01. And $P(\neg D)$ , the probability that any given person doesn’t have the disease, is 0.9999. If you substitute these numbers into Bayes’s Theorem you find

$$ P(D|T) = 0.98\% $$

That is, less than 1% of the people who test positive actually have the disease.

> **_Note_**<br>This assumes that people take the test more or less at random. If only people with certain symptoms take the test we would instead have to condition on the event “positive test _and_ symptoms” and the number would likely be a lot higher.

While this is a simple calculation for a data scientist, most doctors will guess that $P(D|T)$ is approximately 2.

A more intuitive way to see this is to imagine a population of 1 million people. You’d expect 100 of them to have the disease, and 99 of those 100 to test positive. On the other hand, you’d expect 999,900 of them not to have the disease, and 9,999 of those to test positive. Which means that you’d expect only 99 out of (99 + 9999) positive testers to actually have the disease.

# Random Variables

A _random variable_ is a variable whose possible values have an associated probability distribution. A very simple random variable equals 1 if a coin flip turns up heads and 0 if the flip turns up tails. A more complicated one might measure the number of heads observed when flipping a coin 10 times or a value picked from `range(10)` where each number is equally likely.

The associated distribution gives the probabilities that the variable realizes each of its possible values. The coin flip variable equals 0 with probability 0.5 and 1 with probability 0.5. The `range(10)` variable has a distribution that assigns probability 0.1 to each of the numbers from 0 to 9.

We will sometimes talk about the _expected value_ of a random variable, which is the average of its values weighted by their probabilities. The coin flip variable has an expected value of 1/2 (= 0 * 1/2 + 1 * 1/2), and the `range(10)` variable has an expected value of 4.5.

Random variables can be _conditioned_ on events just as other events can. Going back to the two-child example from “Conditional Probability” on page 70, if $X$ is the random variable representing the number of girls, $X$ equals 0 with probability 1/4, 1 with probability 1/2, and 2 with probability 1/4.

We can define a new random variable $Y$ that gives the number of girls conditional on at least one of the children being a girl. Then $Y$ equals 1 with probability 2/3 and 2 with probability 1/3. And a variable $Z$ that’s the number of girls conditional on the older child being a girl equals 1 with probability 1/2 and 2 with probability 1/2.

For the most part, we will be using random variables _implicitly_ in what we do without calling special attention to them. But if you look deeply you’ll see them.

# Continuous Distributions

A coin flip corresponds to a _discrete distribution_ — one that associates positive probability with discrete outcomes. Often we’ll want to model distributions across a continuum of outcomes. (For our purposes, these outcomes will always be real numbers, although that’s not always the case in real life.) For example, the _uniform distribution_ puts _equal weight_ on all the numbers between 0 and 1.

Because there are infinitely many numbers between 0 and 1, this means that the weight it assigns to individual points must necessarily be zero. For this reason, we represent a continuous distribution with a _probability density function_ (pdf) such that the probability of seeing a value in a certain interval equals the integral of the density function over the interval.

> **_Note_**<br>If your integral calculus is rusty, a simpler way of understanding this is that if a distribution has density function $f$, then the probability of seeing a value between $x$ and $x + h$ is approximately $h * f(x)$ if $h$ is small.

The density function for the uniform distribution is just:

In [ ]:
def uniform_pdf(x):
    return 1 if x >= 0 and x < 1 else 0

The probability that a random variable following that distribution is between 0.2 and 0.3 is 1/10, as you’d expect. Python’s `random.random()` is a [pseudo]random variable with a uniform density.

We will often be more interested in the _cumulative distribution function_ (cdf), which gives the probability that a random variable is less than or equal to a certain value. It’s not hard to create the cumulative distribution function for the uniform distribution (Figure 6-1):

In [ ]:
def uniform_cdf(x):
    "returns the probability that a uniform random variable is <= x"
    if x < 0:   return 0    # uniform random is never less than 0
    elif x < 1: return x    # e.g. P(X <= 0.4) = 0.4
    else:       return 1    # uniform random is always less than 1

> **Figure 6-1: _The uniform cdf_**<br><img width="300" src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAq4AAAIVCAIAAACfkOrnAAAACXBIWXMAAAsSAAALEgHS3X78AAAgAElEQVR42u3de3SUhZ344XfGQAIJCXIziQhBSbYWakNSLuIFQrcL4gEXllu8sFAtnt2u1HjkorUHOWiPekQ8rKLFPQLxkiIRL6tYXVxRVERIYD0qlFIbKCbcaggRkhjI/P6Y88tmA8Sg3PM8f3DyvvNOmHznTeaT952ZhCKRSACcEoMHD37vvffq6urOxptaW1t73333Pf/889u3b6+trX355ZdHjhx5tt8jkyZNys/PLykp6datW/3K+fPnP/nkk3/5y19qamrmzZv3q1/9yq7LuS3GCOB7CofD37rN4sWLJ06cGARBKBQ6K76oUCjU6KbOnTt3zpw5gwYNmjBhQqtWrX7wgx+cG3dfoy/z97///e23356VlXXHHXfExsZefvnl9nCkAPAtZs2aVf9wEolEHn300YqKittvv719+/b122RmZp5dX1R+fn5VVVXDNa+99lq7du3+67/+KybmnPq50ejI6GuvvRb9Nzk52b6NFACamwINFxctWrR///7bb7+94THns85FF13UaE1paWnHjh3PsQ44UmlpaSgU0gG0KGEjgFPs8OHDv/3tb9PT0+Pi4rp16zZz5sza2tpG22zevHnSpEkXXXRRbGxscnLyDTfcsGXLlqY/7eLFi8Ph8JIlSxp/k4fDOTk59Yv33ntvOBx+9913CwsL+/XrFx8f37Fjx9zc3NLS0obXGjx4cP2Jj0mTJoXD4ZKSkpKSknA4HA6He/ToUb/lCy+8cPXVVyclJbVt2/ayyy574IEHvvnmm4afKi0trUePHpWVlXfccUdaWlrr1q1nz55dv/7AgQN5eXkXXXRR27ZtMzMzX3755SAIDh06dN9996Wnp7dp06Znz56PP/54cwa7Y8eOqVOnpqent23btmPHjv3797/vvvsabrBy5cqrrroq+iWPGjVq8+bNDS+NTmbVqlWRSCT8/9ldcVQAOMEikUhubu77778/fPjwxMTE119//aGHHtq9e/fTTz9dv80f/vCH0aNHHz58eMSIET179vzrX/+6fPny119//Z133unTp0/Tn/+oz0U4cuWCBQteffXV6667Licn56OPPlq6dOn//M//bNy4sXXr1kdea9SoUT169Hj00UeDIMjLywuCoP7cx9133/3AAw907tz5xhtvTEhIWLFixd133/3mm2++9dZbrVq1qv8833zzTU5Ozr59+4YNG5aYmHjxxRdHL6qtrf3Zz35WXl4+atSompqagoKCMWPGvPHGG48//nhRUdHw4cNjY2NfeOGF2267rXPnzuPGjWviC1+/fv3QoUPLy8sHDRo0ZsyYgwcPfvbZZ7Nnz77nnnuiGxQWFo4fPz4uLm78+PEpKSmrV68eOHDgZZddVv8ZcnJyQqHQ4sWLt23bdu+999pXaVk/mIATqHv37uFweNu2bUdeNGjQoFAo9JOf/KS8vDy65sCBAz179jzvvPN27twZXfPVV1+1b9++c+fOmzZtqr/ip59+mpCQkJWV1cT/u2jRolAotGTJkkbrQ6FQTk5O/WL0mQ1JSUmffvpp/crrr78+FAq98MILDW9qOBxu9HX16NGj4ZoPP/wwFAp17959165d0TWHDh0aMWJEKBT67W9/2/CKoVDoZz/72cGDBxt9wlAoNHLkyG+++Sa6ZvXq1dHb1q9fv4qKiujKL774onXr1n369Gnia6+pqUlLSwuHwwUFBQ3Xf/nll9EPKisrO3To0Lp166KiovpL8/LyQqFQozvryC8cznkOf8Gp9uCDD9b/Vt22bdsbbrihrq6uqKgouiY/P7+iomL27NkNn6Lfq1evW265ZcOGDZs2bToht2Hq1Km9evWqX/zFL34RBMG6deuO65NEj2Tcc889Xbp0ia4577zz5s6dGw6H/+M//qPRYYm5c+e2adPmyMMVjz76aP3xgyuvvDItLa2ysvLBBx9MTEyMruzRo8fAgQM/++yzJl75/J//+Z/btm0bOXLkhAkTGq5PTU2NfvDKK6+Ul5dff/31WVlZDc8I1P8v4AQBcIpEjwo0XNO1a9cgCMrLy6OLa9asCYJg48aNjY5RR58rsGnTpksvvfT734ymb0MzFRcXh0KhIUOGNFyZnp5+4YUXlpSUVFZWtmvXLroyLi7uRz/60ZGfoX379g2fdhB98N62bVt2dnbDlRdeeGFtbe3OnTtTUlKOeks++uijIAiuueaaJm5qEASDBg1quDIxMTEzM/O9996zWyIFgFOq0W+i0efkHz58OLr4t7/9LQiCp5566qgZceDAgRNyGxq+0PHI29BMFRUVQRAc+fCckpKyY8eOffv21adA/WGDRpKSkhr/SIqJCYKg/ooNVx755Mp6+/btixZD0zf1ggsuaLTeKwVACsAZJ/ro+Mknn/Tu3fu4rhh9uvuhQ4eOfIw8qTe1rKys/mmAUWVlZY0e5k/2GytFy2bHjh1N39Rdu3Y1Wr9z5067HHiuAJxZom9v9x2OWp9//vlBEGzfvr3hyvXr15+8mxp9GuOqVasarty6deuOHTt69OhxKk/DR4f2xhtvHGuD6BmHRje1oqJi48aNZ8v7P4IUgJZi8uTJ7du3nz17dqMn8dXV1TV6JGukb9++4XD4+eefr3+XwK+++mr69Okn76b+/Oc/D4Lgvvvu27t3b3TN4cOH77zzzkgkcvPNN5/KoY0YMSItLe3VV1/9/e9/33B9/XGC66677vzzz3/++efrn54ZBMG99967f/9+uxw4QQAnXhPPdf/WPwDWoUOHwsLCUaNGDRgw4Kc//ekPf/jDUCj017/+dc2aNeXl5QcPHjzWFaPvRPTMM89kZmYOHz58//79b7zxxqBBgzZu3HjCv4r638WnT5/+0EMP9e7de8yYMW3btn3jjTc+++yzq666atq0aSdjesfSqlWrZcuW/cM//MP111//u9/9rn///tXV1Zs2bfrv//7v6DMM4uPjFy5cOH78+Kuuumr8+PHJycnvv//+Z599dvXVVx95AMYfacNRAeB7OfIP+TR90ZErhwwZ8sknn/zrv/5rSUnJ7373u0WLFn3++ed///d/3+hX3iM99dRTd95558GDBxcsWLB69epf/epXzz77bPNvXtObHfVaDzzwQEFBQXp6en5+/r//+78HQXD//fc3+jsFTUzjO4/oSNnZ2Rs3bvyXf/mXbdu2zZs377nnntu/f/+cOXPqN/inf/qnP/zhD9nZ2S+88MLChQs7der00Ucf9ejR48gv0ykDWtxPLf0LAI4KAABSAACQAgCAFAAApAAA0AKc5vcV2Lt375tvvpmWlnbknywDAL6nqqqqkpKSoUOHdurU6Zgbnd6/kXzki54BgBPr2WefbeKx+DQfFUhLS4vexBPyd1dbmry8vHnz5pmD0Zme0Rkdx7Jp06Ybb7wx+mh7LKc5BaLnBS699NKsrCx32PFKSkoyN6MzPaMzOpr5aHssnjYIAC2aFAAAKQAASAHOOrm5uYZgdKZndEbH93Sa/zJhcXFxdnZ2UVGR54MAwGl5nHVUAABaNCkAAFIAAJACAIAUAACkAAAgBQAAKQAASAEAQAoAAFIAAJACAIAUAACkAAAgBQAAKQAASAEAQAoAAFIAADgHU+DAgQOzZs0aNmxYhw4dwuHwkiVLmth48eLF4SPs3r3bxAHgjBLT/E337NkzZ86c7t27Z2Zmrlq1KhQKfetV5syZ06NHj/rFpKQkEweAszUFUlNTd+7c2aVLl6Kior59+zbnKtdcc01WVpYpA8AZ6zhOELRu3bpLly5BEEQikWZeJRKJVFZWHj582KAB4KxPge8gJycnKSkpPj7+uuuu27p1q3EDwJkm5iR93vj4+MmTJ+fk5CQmJq5fv/6RRx4ZOHBgcXFx165dDR0Azv0UGDt27NixY6Mfjxw5cujQoVdfffX999//xBNPGDoAnDlO0fsKXHHFFf3791+5cqWJA0CLOCpwpK5du27ZsuWoF+Xl5TV8nWFubm5ubq77BgCOV0FBQUFBQf1iRUXFGZQCX3zxRefOnY960bx587zmEAC+v0a/ThcXF2dnZzd9lRN2gmDnzp2bN28+dOhQdHHPnj0NL12xYkVxcfGwYcPcSQBwRjm+owKPPfbYvn37SktLgyB49dVXt2/fHgTB1KlTExMTZ86cmZ+fX1JS0q1btyAIBg4cmJWVlZ2dnZSUVFxc/PTTT3fr1u3uu+82cQA4i1Ng7ty527ZtC4IgFAq99NJLy5cvD4VCEydOTExMDIVCDd+KeMKECa+//vpbb7118ODB1NTUW2+9ddasWcc6QQAAnC6h5r914MkQPYdRVFTkuQIAcFoeZ/2RYgBo0aQAAEgBAEAKAABSAACQAgCAFAAApAAAIAUAACkAAEgBAEAKAABSAACQAgCAFAAApAAAIAUAACkAAEgBAEAKAABSAACQAgCAFAAApAAAIAUAACkAAEgBAEAKAABSAACQAgCAFAAApAAAIAUAACkAAEgBAEAKAABSAACQAgCAFAAApAAAIAUAACkAAEgBAEAKAABSAACQAgCAFAAApAAAIAUAACkAAEgBAEAKAABSAACkAAAgBQAAKQAASAEAQAoAAFIAAJACAIAUAACkAAAgBQAAKQAASAEAQAoAAFIAAJACAIAUAACkAAAgBQAAKQAASAEAQAoAAFIAAJACAIAUAACkAAAgBQAAKQAASAEAQAoAAFIAAJACAIAUAACkAAAgBQAAKQAASAEAQAoAAFIAADilKXDgwIFZs2YNGzasQ4cO4XB4yZIlTW+/b9++KVOmdO7cOSEhYciQIRs2bDBuADiLU2DPnj1z5sz54x//mJmZGQRBKBRqYuO6urprr722oKBg6tSpDz300O7duwcPHrx161YTB4AzSkzzN01NTd25c2eXLl2Kior69u3b9MaFhYVr1qwpLCwcPXp0EATjxo3LyMiYNWvWc889Z+gAcFYeFWjdunWXLl2CIIhEIt+6cWFhYXJycrQDgiDo1KnTuHHjXnnlldraWkMHgLMyBY7Lhg0bsrKyGq7p27fvwYMHt2zZYugAcO6nQFlZWUpKSsM10cXS0lJDB4BzPwWqq6tjY2MbromLiwuCoKqqytAB4MwRc5I+b5s2bWpqahrFQXS9oQMt0ILX3v/P//nAHDjFKkq/PG0pkJKS0uhcQFlZWRAEqampR26cl5eXlJRUv5ibm5ubm+v+A84ZMxa/9NAX44JDbUN1rUyDkyryeU2wqcGv4tXf/kz/k5UCmZmZq1evjkQi9W8/sHbt2vj4+IyMjCM3njdvXqPnGAKcYx1w0df/tOWBZ+NaxxgIp1JxcXF2dnbT25yw5wrs3Llz8+bNhw4dii6OGTNm165dy5cvjy7u3bt32bJlI0aMaNVKEQM6AM4gx7dfPvbYY/v27Yse+X/11Ve3b98eBMHUqVMTExNnzpyZn59fUlLSrVu3aAoMGDBg8uTJn3/+eceOHRcsWBCJRGbPnm3igA6AszgF5s6du23btiAIQqHQSy+9tHz58lAoNHHixMTExFAo1PCtiMPh8IoVK6ZNmzZ//vyqqqp+/frl5+enp6ebOKAD4IwSas5bB5480XMYRUVFnisA6AA4LY+z/kgxgA6gRZMCADoAKQCADkAKAKADkAIA6ACkAAA6ACkAgA5ACgCgA5ACAOgApAAAOgApAIAOQAoA6AAdgBQA0AE6ACkAoANACgDoAJACADoApACADgApAKADQAoA6ACQAgA6AKQAgA4AKQCgA0AKAOgAkAIAOgCkAIAOACkAoANACgDoAJACADoApACADgApAKADQAoA6ACQAgA6AKQAgA4AKQCgA0AKAOgAkAIAOgCkAIAOACkAoANACgDoAJACADoApACADgApAKADQAoA6ACQAoAO0AEgBQAdoANACgA6AJACgA4ApACgAwApAOgAQAoAOgCQAoAOAKQAoAMAKQDoAJACADoApACADgApAKADQAoA6ACQAgA6AKQAgA4AKQCgA0AKAOgAkAIAOgCkAIAOACkAoANACgDoAJACADoApACADgApAKADQAoA6ACQAgA6AKQAgA4AKQCgA0AKAOgAkAIAOgCkAIAOACkAoANACgA6QAeAFAB0ACAFAB0ASAFABwBSANABgBQAdAAgBQAdAEgBQAcAUgDQAYAUAHQAIAUAHQBIAUAHAFIA0AGAFAB0ACAFAB0AnNIUqKmpmTFjRmpqatu2bQcMGLBy5cpjbbl48eLwEXbv3m3coAOAM81xfNNOmjTpxRdfzMvLS09PX7Ro0fDhw995550rrrjiWNvPmTOnR48e9YtJSUnGDToAOFtT4OOPP166dOnDDz98xx13BEFw00039e7de/r06R988MGxrnLNNddkZWUZMegA4EzW3BMEhYWFMTExU6ZMiS7GxsbefPPNa9as+fLLL491lUgkUllZefjwYVMGHQCc9SmwYcOGjIyMhISE+jV9+/YNgmDjxo3HukpOTk5SUlJ8fPx11123detWswYdAJyBmvsNXFZWlpKS0nBNdLG0tPTIjePj4ydPnpyTk5OYmLh+/fpHHnlk4MCBxcXFXbt2NXHQAcBZmQJVVVWxsbEN18TFxUXXH7nx2LFjx44dG/145MiRQ4cOvfrqq++///4nnnjCxEEHAGeU5p4gaNOmTU1NTcM11dXV0fXfet0rrriif//+Tbz4ENABwJmeAikpKY3OBZSVlQVBkJqa2pyrd+3atby83LhBBwBnmuZ+M/fp02fVqlWVlZXt2rWLrlm7dm0QBJmZmc25+hdffNG5c+djXZqXl9fwXQdyc3Nzc3PdN6ADgONVUFBQUFBQv1hRUfHt14k0z9q1a0Oh0MMPPxxdrK6u7tmz5+WXXx5dLCsr27RpU21tbXRx9+7dDa/7+uuvh0Kh22+//chPW1RUFARBUVFRBDiDTV+0PPhNzEV546tqak0DziLNeZxtbtr369dv7Nixd9111+7duy+55JIlS5Zs37590aJF0UtnzpyZn59fUlLSrVu3IAgGDhyYlZWVnZ2dlJRUXFz89NNPd+vW7e677xZr4HgAcLaeIAiCID8//ze/+c0zzzxTXl7+4x//+LXXXrvyyiujF4VCoVAoVL/lhAkTXn/99bfeeuvgwYOpqam33nrrrFmzmjhBAOgA4HQJRSKR0/jfFxcXZ2dnFxUVeYti0AHAaXmc9UeKAR0ALZoUAHQASAEAHQBSAEAHgBQAdIAOACkA6AAdAFIA0AGAFAB0ACAFAB0ASAFABwBSANABgBQAdAAgBQAdAEgBQAcAUgDQAYAUAHQAIAUAHQBIAUAHAFIA0AGAFAB0ACAFAB0ASAFABwBSANABgBQAdAAgBQAdAEgBQAcAUgDQAYAUAHQAIAUAHQBIAUAHAFIA0AGAFAB0ACAFAB0ASAFABwBSANABgBQAdAAgBUAH6ABACoAO0AGAFAAdACAFQAcASAHQAQBSAHQAgBQAHQAgBUAHAEgB0AEAUgB0AIAUAB0AIAVABwBIAdABAFIAdACAFAAdACAFQAcASAHQAYAUAHQAIAUAHQBIAUAHAFIA0AGAFAB0ACAFAB0ASAFABwBSANABgBQAdAAgBQAdAEgBQAcAUgDQAYAUAHQAIAUAHQBIAUAHAFIAdIAOAKQA6AAdAEgB0AEAUgB0AIAUAB0AIAVABwBIAdABAFIAdACAFAAdACAFQAcASAHQAQBSAHQAgBQAHQAgBUAHAEgB0AEAUgB0AIAUAB0AIAVABwBIAdABAFIAdACAFAAdACAFQAcASAHQAQBSAHQAwKlKgZqamhkzZqSmprZt23bAgAErV65sYuN9+/ZNmTKlc+fOCQkJQ4YM2bBhg1mjAwDO7hSYNGnSvHnzbrrppvnz55933nnDhw//4IMPjrplXV3dtddeW1BQMHXq1Iceemj37t2DBw/eunWrcaMDAM40zf2p9/HHHy9duvThhx++4447giC46aabevfuPX369KPWQGFh4Zo1awoLC0ePHh0Ewbhx4zIyMmbNmvXcc8+ZODoA4Kw8KlBYWBgTEzNlypToYmxs7M0337xmzZovv/zyqBsnJydHOyAIgk6dOo0bN+6VV16pra01cXQAwFmZAhs2bMjIyEhISKhf07dv3yAINm7ceNSNs7KyGq7p27fvwYMHt2zZYuLoAICzMgXKyspSUlIarokulpaWfs+NQQcAnAUpUFVVFRsb23BNXFxcdP2RG1dXVzd/Y9ABAKdRc38ItmnTpqamptHjfXT999w4CILs+UNCyX4cc+pEYssv+nqsDgA4jhRISUlpdHi/rKwsCILU1NTvuXEQBOe/2yEmLu5/r/6jH6f+ONN9w8nTOb7Dwl9P1gHAuaegoKCgoKB+saKi4oSlQJ8+fVatWlVZWdmuXbvomrVr1wZBkJl5lMfszMzM1atXRyKRUChUv3F8fHxGRsZRP/nKFwsbPc0QAPgOcnNzc3Nz6xeLi4uzs7ObvkpznyswZsyYw4cPL1y4MLpYU1OzaNGiAQMGXHjhhUEQ7Ny5c/PmzYcOHarfeNeuXcuXL48u7t27d9myZSNGjGjVqpU7CQDOKM09KtCvX7+xY8feddddu3fvvuSSS5YsWbJ9+/ZFixZFL505c2Z+fn5JSUm3bt2iKTBgwIDJkyd//vnnHTt2XLBgQSQSmT17tnEDwNmaAkEQ5Ofn/+Y3v3nmmWfKy8t//OMfv/baa1deeWX0olAoVH8uIAiCcDi8YsWKadOmzZ8/v6qqql+/fvn5+enp6cYNAGeaUCQSOY3/ffQcRlFRkecKAMBpeZz1R4oBoEWTAgAgBQAAKQAASAEAQAoAAFIAAJACAIAUAACkAAAgBQAAKQAASAEAQAoAAFIAAJACAIAUAACkAAAgBQAAKQAASAEAQAoAAFIAAJACAIAUAACkAAAgBQAAKQAASAEAQAoAAFIAAJACAIAUAACkAAAgBQAAKQAASAEAQAoAAFIAAJACAIAUAACkAAAgBQAAKQAASAEAQAoAAFIAAJACAIAUAACkAAAgBQAAKQAASAEAkAIAgBQAAKQAACAFAAApAABIAQBACgAAUgAAkAIAgBQAAKQAACAFAAApAABIAQBACgAAUgAAkAIAgBQAAKQAACAFAAApAABIAQBACgAAUgAAkAIAgBQAAKQAACAFAAApAABIAQBACgAAUgAAkAIAgBQAAKQAACAFAAApAABIAQDglKbAvn37pkyZ0rlz54SEhCFDhmzYsKGJje+9997w/9WmTRvjBoAzTUwzt6urq7v22ms/+eST6dOnd+zYccGCBYMHDy4qKurZs2cT13ryyScTEhKiH5933nnGDQBnawoUFhauWbOmsLBw9OjRQRCMGzcuIyNj1qxZzz33XBPXGjNmTIcOHUwZAM5YzT1BUFhYmJycHO2AIAg6deo0bty4V155pba2tuljCfv3749EIgYNAGd3CmzYsCErK6vhmr59+x48eHDLli1NXOviiy9u3759YmLiTTfdtHv3buMGgDNNc08QlJWVDR48uOGalJSUIAhKS0t79ep15PYdOnS47bbbLr/88tjY2Pfee+/xxx//+OOP169f365dO0MHgDMoBSKRSE1NTRNbxMXFBUFQXV0dGxt75PqqqqqjXmvq1Kn1H48aNapfv3433HDDggULZsyYYegAcOYIv/vuu22bFD0F0KZNm0bFUF1dHV3fnP8mNzc3OTn57bffNnEAOLOOClx66aWLFy9uYovk5OQgCFJSUkpLSxuuLysrC4IgNTW1mf9T165dv/rqq6NelJeXl5SU1LAbcnNz3TcAcLwKCgoKCgrqFysqKr49BS644IKJEyd+63aZmZmrV6+ORCKhUCi6Zu3atfHx8RkZGc25ZZFIpKSkJDs7+6iXzps3r9FzEgGA76DRr9PFxcXHevCt19xXEIwZM2bXrl3Lly+PLu7du3fZsmUjRoxo1apVdM327ds3b95cv/2ePXsaXv2JJ57Yu3fvsGHD3EkAcEZp7isIxowZM2DAgMmTJ3/++efRdxuMRCKzZ8+u32DixInvvfdeXV1ddLF79+4TJkzo3bt3XFzc+++/v3Tp0j59+tx6660mDgBnZQqEw+EVK1ZMmzZt/vz5VVVV/fr1y8/PT09Pr98gFArVnzsIguDGG2/88MMPX3zxxerq6rS0tBkzZvz617+OvugAADj7UiAIgvbt2z/11FNPPfXUUS995513Gi4uXLjQcAHgzOePFAOAFAAApAAAIAUAACkAAEgBAEAKAABSAACQAgCAFAAApAAAIAUAACkAAEgBAEAKAABSAACQAgCAFAAApAAAIAUAACkAAEgBAEAKAABSAACQApwABQUFhmB0pmd0RocU8L2B0Zme0RkdUgAAkAIAgBQAAI5DzOn976uqqoIg2LRpk3viO6ioqCguLjYHozM9ozM6jiX6CBt9tD2myGn17LPPup8A4KR69tlnm3gsDkUikdN44/bu3fvmm2+mpaW1adPGXQUAJ1ZVVVVJScnQoUM7dep0rG1OcwoAAKeXpw0CgBQAAKQAACAFTrqdO3fOnDkzJyenXbt24XD43Xff/darfPnll+PGjTv//POTkpL+8R//8S9/+UtLvsP27ds3ZcqUzp07JyQkDBkyZMOGDU1sfO+994b/rxby9MyampoZM2akpqa2bdt2wIABK1euPFEjNb2GFi9eHD7C7t27W+bcDhw4MGvWrGHDhnXo0CEcDi9ZsuQEfi+bnh3vqNatW/dv//ZvvXr1SkhI6N69+/jx4//0pz99h73uVL+vwObNmx966KGMjIzLLrtszZo1oVCo6e2//vrrnJycysrKX//61zExMfPmzRs0aNDGjRs7dOjQAu/1urq6a6+99pNPPpk+fXrHjh0XLFgwePDgoqKinj17NnGtJ598MiEhIfrxeeed1xIGNWnSpBdffDEvLy89PX3RokXDhw9/5513rpWnmIEAAAb6SURBVLjiihM1UtNraM6cOT169KhfTEpKaplz27Nnz5w5c7p3756Zmblq1aqmf7jZ8b7P9Ox49R588ME1a9aMHTv2sssuKysre+yxx7Kysj766KNevXod3153it9IoLKysry8PBKJLFu2LBQKvfvuu01v/+CDD4ZCofXr10cXN2/eHBMTc/fdd0dapKVLl4ZCoRdffDG6uGfPnvPPP//6668/1vazZs0KhUJ/+9vfWtSU1q5dGwqF5s6dG12srq7u2bPnwIEDT8hITa+hRYsWhUKhoqKiCJFITU3Nrl27IpHI+vXrQ6HQkiVLTuD3sunZ8Y7qww8/rK2trV/805/+FBcXd+ONNx7vXneqTxAkJCS0b9+++dsXFhb269cvOzs7uvh3f/d3P/3pT1944YWWGc6FhYXJycmjR4+OLnbq1GncuHGvvPJKbW1t079/7N+/v+W8arSwsDAmJmbKlCnRxdjY2JtvvnnNmjVffvnliRqp6TUUTfzDhw+38BMrrVu37tKlS3QgJ+l72fTseI1cfvnlMTH/e3S/Z8+eP/zhDzdv3ny8e90Z/bTBurq6Tz755Cc/+UnDlX379v3zn/984MCBFnivb9iwISsrq9E0Dh48uGXLliaudfHFF7dv3z4xMfGmm25qCafTNmzYkJGRUX9OJDqlIAg2btx4okZqeg3l5OQkJSXFx8dfd911W7duDThp38vY8b41j3bt2nWstxJqYq87o1Pgq6+++uabb1JSUhqujC6Wlpa2wLu5rKzsuKbRoUOH2267beHChS+++OItt9yydOnSq666qrKy0pS+80jtYw3Fx8dPnjx5wYIFL7/88vTp099+++2BAwfu2LHDT+QT/r2MHa85nnvuudLS0vHjxx/vXnfCnjYYPdnTxAZxcXHH+zmjfz4hNjb2yM/zLX9Z4SxMueZMr7q6+rimMXXq1PqPR40a1a9fvxtuuGHBggUzZsw4h78Zqqqqmj+l4x3pOe+4pjd27NixY8dGPx45cuTQoUOvvvrq+++//4knnvBDuWl2vO/DjndUmzdv/uUvfzlw4MB//ud/Pt697oQdFXj33XfbNuk7HPiKvvKt0WNkdXV1/UXnjGZOr02bNt9nGrm5ucnJyW+//fa5/f1wXFP6niNt4dNr5Iorrujfv3/TL93EjnfC2fGCINi5c+e11157/vnnFxYWHuv1F03sdSfsqMCll166ePHiJjZITk4+3s/ZoUOH2NjYsrKyRoc4giBITU09l+7FZk4vJSWl0fHD451G165dv/rqq3P7W+K4pvT9R9qSp3fUHczZ7lMwZ+x4DVVUVFxzzTX79+9fvXp1Ew+1Tex1JywFLrjggokTJ57YLy8cDv/oRz9at25dw5Vr16695JJL4uPjz6U7spnTy8zMXL16dSQSqY++tWvXxsfHZ2RkNOd/iUQiJSUl9S/HOFf16dNn1apVlZWV7dq1q59SdHonfKQtfHpH+uKLLzp37uyR6WR/L2PHa/ib/YgRI7Zu3bpy5cof/OAH322vO+OeNrh9+/aGL4QYM2bMunXrioqKoot//OMf33nnnfqzRC3NmDFjdu3atXz58uji3r17ly1bNmLEiFatWh11env27Gl49SeeeGLv3r3Dhg0756d0+PDhhQsXRhdramoWLVo0YMCACy+8MAiCnTt3bt68+dChQ80caQvcx5o/vUY72IoVK4qLi8/5Hey7seOdwOnZ8eodPnx4/Pjxa9euXbZsWf/+/b/zXnca/kjxfffdFwTBZ599tnTp0p///OdpaWlBENxzzz3RSwcPHvzee+/V1dVFF7/++us+ffpUVlbeeeedMTExjzzySCQS2bhxY8eOHVvgvV5XV3fllVd++umn06ZNi75X1I4dO9atW5eenn7U6bVt23bChAm9e/eOi4t7//33ly5dmpmZ+cEHH3yHp3CeXcaPH//SSy/l5eVdcsklS5YsWb9+/dtvv33llVcGQTBp0qT8/PySkpJu3bo1Z6QtUPOnl56enpWVlZ2dnZSUVFxc/PTTT1944YXr1q1rsb+fPfbYY/v27SstLX3yySdHjx4dPZQyderUxMREO94JnJ4dr97tt98+f/78ESNGNPoN+cYbbzy+H3en/t2RQqFQOByu/zf6Qf2lgwcPbrgYiUR27NgxduzYpKSkdu3ajRw58s9//nNLfm+p8vLyW265pVOnTvHx8Tk5OY3eb6vR9H7xi1/06tUrMTGxdevWGRkZd91119dff90SplRdXT1t2rSUlJS4uLj+/fu/9dZb9RdNmjQpHA5v27atmSNtgZo/vXvuuadPnz7t27dv3bp1WlraL3/5y927d7fk0aWlpdX/TKv/KRcdlx3vBE7PjtfoZ37o/6p/FGj+Xvf/AA5rXGtSNHTRAAAAAElFTkSuQmCC" />

# The Normal Distribution

The normal distribution is the king of distributions. It is the classic bell curve–shaped distribution and is completely determined by two parameters: its mean $\mu$ (mu) and its standard deviation $\sigma$ (sigma). The mean indicates where the bell is centered, and the standard deviation how “wide” it is.

It has the distribution function:

$$ f(x|\mu,\sigma) = \frac{1}{\sqrt{2\mu}\sigma} exp\left( -\frac{(x-\mu)^2}{2\sigma^2} \right) $$

which we can implement as:

In [ ]:
def normal_pdf(x, mu=0, sigma=1):
    sqrt_two_pi = math.sqrt(2 * math.pi)
    return (math.exp(-(x-mu) ** 2 / 2 / sigma ** 2) / (sqrt_two_pi * sigma))

In Figure 6-2, we plot some of these pdfs to see what they look like:

In [ ]:
xs = [x / 10.0 for x in range(-50, 50)]
plt.plot(xs,[normal_pdf(x,sigma=1) for x   in xs],'-',label='mu=0,sigma=1')
plt.plot(xs,[normal_pdf(x,sigma=2) for x   in xs],'--',label='mu=0,sigma=2')
plt.plot(xs,[normal_pdf(x,sigma=0.5) for   x in xs],':',label='mu=0,sigma=0.5')
plt.plot(xs,[normal_pdf(x,mu=-1)   for x   in xs],'-.',label='mu=-1,sigma=1')
plt.legend()
plt.title("Various Normal pdfs")
plt.show()

> **Figure 6-2: _Various normal pdfs_**<br><img width="300" src="data:image/png;base64, ..." />

When $\mu = 0$ and $\sigma = 1$, it’s called the _standard normal distribution_ . If $Z$ is a standard normal random variable, then it turns out that:

$$ X = \sigma Z + \mu $$

is also normal but with mean $\mu$ and standard deviation $\sigma$ . Conversely, if $X$ is a normal random variable with mean $\mu$ and standard deviation $\sigma$,

$$ Z = \frac{(X - \mu)}{\sigma} $$

is a standard normal variable.

The cumulative distribution function for the normal distribution cannot be written in an “elementary” manner, but we can write it using Python’s `math.erf`:

In [ ]:
def normal_cdf(x, mu=0,sigma=1):
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2
Again, in Figure 6-3, we plot a few:
xs = [x / 10.0 for x in range(-50, 50)]
plt.plot(xs,[normal_cdf(x,sigma=1) for x in xs],'-',label='mu=0,sigma=1')
plt.plot(xs,[normal_cdf(x,sigma=2) for x in xs],'--',label='mu=0,sigma=2')
plt.plot(xs,[normal_cdf(x,sigma=0.5) for x in xs],':',label='mu=0,sigma=0.5')
plt.plot(xs,[normal_cdf(x,mu=-1) for x in xs],'-.',label='mu=-1,sigma=1')
plt.legend(loc=4) # bottom right
plt.title("Various Normal cdfs")
plt.show()

> **Figure 6-3: _Various normal cdfs_**<br><img width="300" src="data:image/png;base64, ..." />

Sometimes we’ll need to invert `normal_cdf` to find the value corresponding to a specified probability. There’s no simple way to compute its inverse, but `normal_cdf` is continuous and strictly increasing, so we can use a binary search:

In [ ]:
def inverse_normal_cdf(p, mu=0, sigma=1, tolerance=0.00001):
    """find approximate inverse using binary search"""
    # if not standard, compute standard and rescale
    if mu != 0 or sigma != 1:
        return mu + sigma * inverse_normal_cdf(p, tolerance=tolerance)
    low_z, low_p = -10.0, 0            # normal_cdf(-10) is (very close to) 0
    hi_z, hi_p = 10.0, 1               # normal_cdf(10) is (very close to) 1
    while hi_z - low_z > tolerance:
        mid_z = (low_z + hi_z) / 2     # consider the midpoint
        mid_p = normal_cdf(mid_z)      # and the cdf's value there
        if mid_p < p:
            # midpoint is still too low, search above it
            low_z, low_p = mid_z, mid_p
        elif mid_p > p:
            # midpoint is still too high, search below it
            hi_z, hi_p = mid_z, mid_p
        else:
            break
    return mid_z

The function repeatedly bisects intervals until it narrows in on a $Z$ that’s close enough to the desired probability.

# The Central Limit Theorem

One reason the normal distribution is so useful is the _central limit theorem_, which says (in essence) that a random variable defined as the average of a large number of independent and identically distributed random variables is itself approximately normally distributed.

In particular, if $x_1, \cdots, x_n$ are random variables with mean $\mu$ and standard deviation $\sigma$, and if $n$ is large, then:

$$ \frac{1}{n} \left( x_1 + \cdots + x_n \right) $$

is approximately normally distributed with mean $\mu$ and standard deviation $\sigma / \sqrt{n}$. Equivalently (but often more usefully),

$$ \frac{(x_1 + \cdots + x_n) - \mu n}{\sigma\sqrt{n}} $$

is approximately normally distributed with mean 0 and standard deviation 1.

An easy way to illustrate this is by looking at _binomial_ random variables, which have two parameters $n$ and $p$. A Binomial(n,p) random variable is simply the sum of $n$ independent Bernoulli(p) random variables, each of which equals 1 with probability $p$ and 0 with probability $1 - p$ :

In [ ]:
def bernoulli_trial(p):
    return 1 if random.random() < p else 0
def binomial(n, p):
    return sum(bernoulli_trial(p) for _ in range(n))

The mean of a Bernoulli(p) variable is $p$, and its standard deviation is $\sqrt{p (1 - p)}$ . The central limit theorem says that as $n$ gets large, a Binomial(n,p) variable is approximately a normal random variable with mean $\mu = n p$ and standard deviation $\sigma = \sqrt{n p (1 - p)}$. If we plot both, you can easily see the resemblance:

In [ ]:
def make_hist(p, n, num_points):
    data = [binomial(n, p) for _ in range(num_points)]
    # use a bar chart to show the actual binomial samples
    histogram = Counter(data)
    plt.bar([x - 0.4 for x in histogram.keys()],
            [v / num_points for v in histogram.values()],
            0.8,
            color='0.75')
    mu = p * n
    sigma = math.sqrt(n * p * (1 - p))
    # use a line chart to show the normal approximation
    xs = range(min(data), max(data) + 1)
    ys = [normal_cdf(i + 0.5, mu, sigma) - normal_cdf(i - 0.5, mu, sigma)
          for i in xs]
    plt.plot(xs,ys)
    plt.title("Binomial Distribution vs. Normal Approximation")
    plt.show()

For example, when you call `make_hist(0.75, 100, 10000)`, you get the graph in Figure 6-4.

> **Figure 6-4: _The output from make_hist_**<br><img width="300" src="data:image/png;base64, ..." />

The moral of this approximation is that if you want to know the probability that (say) a fair coin turns up more than 60 heads in 100 flips, you can estimate it as the probability that a Normal(50,5) is greater than 60, which is easier than computing the Binomial(100,0.5) cdf. (Although in most applications you’d probably be using statistical software that would gladly compute whatever probabilities you want.)

# For Further Exploration

- `scipy.stats` contains pdf and cdf functions for most of the popular probability distributions.
- Remember how, at the end of Chapter 5, I said that it would be a good idea to study a statistics textbook? It would also be a good idea to study a probability textbook. The best one I know that’s available online is _Introduction to Probability_.